In [2]:
import datetime
import time
import ccxt 
import pandas as pd 

def date_to_timestamp(date,utc=False):
    """
    str형태의 date를 timestamp로 만들어주기
    :params (str or datetime) date : '%Y-%m-%d %H:%M:%S'형태의 데이터. ex)'2023-01-18 23:00:00'
    :params bool utc : True로 설정할시에 date를 utc 시간이라고 생각
    :return timestamp시간(단위 ms) ex)1674050400000
    :rtype int
    
    ex) date_to_timestamp('2023-01-18 23:00:00') -> 1674050400000
    """
    if type(date) == str: # str인 경우 datetime으로 변환해주기
        dt = datetime.datetime.strptime(date,'%Y-%m-%d %H:%M:%S')
    else: # datetime으로 들어온경우
        dt = date
        
    # time.mktime은 local 타임 기준으로 timestamp를 변경함. 따라서 utc일 경우 +9시간해서 한국시간으로 설정해줘야함
    if utc:
        dt = date + datetime.timedelta(hours = 9)
    
    ts = time.mktime(dt.timetuple()) 
    
    return int(ts*1000)

def make_csv_data(coin_name,period,start_time,end_time):
    """
    coin이름, 수집하고 싶은 봉의 기준 기간, 시작, 끝 시간을 지정해주면 그 기간까지의 데이터를 수집하여 csv 파일로 반환
    :params str coin_name : 코인이름 ex)"BTC/USDT"
    :params str period : 수집기준기간 ex) "1m"
    :params str start_time : 수집 시작 시간 ex) '2022-01-01 00:00:00' (한국시간기준)
    :params str end_time : 수집 끝 시간 ex) '2023-01-01 00:00:00' (한국시간기준)
    :return None
    :rtype None
    
    f'{coin_name}_{period}_{start_time}_{end_time}.csv' 파일로 저장됨
    """
    
    # 입력 end_time은 한국시간기준이므로 utc기준으로 변경해주기. pd.to_datetime 이 UTC 시간으로 반환하기때문에 UTC 시간 기준으로 체크를 해줘야함 (UTC에서 KST로 변경은 제일 마지막에 함)
    utc_end_time = datetime.datetime.strptime(end_time,'%Y-%m-%d %H:%M:%S') - datetime.timedelta(hours = 9)
    
    binance = ccxt.binanceusdm()
    btc_ohlcv = binance.fetch_ohlcv(coin_name,period,since=date_to_timestamp(start_time))

    df = pd.DataFrame(btc_ohlcv, columns=['datetime', 'open', 'high', 'low', 'close', 'volume'])
    df['datetime'] = pd.to_datetime(df['datetime'], unit='ms')
    df.set_index('datetime', inplace=True)

    while len(df)==0:  # 정기 점검이 있는 시간대에는 조회를 해도 결과가 나오지 않음. 500개씩 조회되므로 데이터가 조회될때까지 500개씩 건너뛰기
        if 'd' in period:
            start_time=datetime.datetime.strptime(start_time,'%Y-%m-%d %H:%M:%S') + datetime.timedelta(days=int(f"{period[:-1]}*500")) 
        elif 'm' in period:
            start_time=datetime.datetime.strptime(start_time,'%Y-%m-%d %H:%M:%S') + datetime.timedelta(minutes=int(f"{period[:-1]}*500")) 
        else:
            assert False, '일봉과 분봉만 조회 가능합니다.'

        btc_ohlcv = binance.fetch_ohlcv(coin_name,period,since=date_to_timestamp(start_time))

        df=pd.DataFrame(btc_ohlcv,columns=['datetime','open','high','low','close','volume'])
        df['datetime']=pd.to_datetime(df['datetime'],unit='ms')
        df.set_index('datetime',inplace=True)
    
    total_df = df
    
    check_count = 0
    
    while True:
        
        check_count+=1
        if check_count%10 == 0:
            print(total_df.index[-1])
        
        if utc_end_time <= df.index[-1]:
            break
        
        if 'd' in period:
            time_later=df.index[-1] + datetime.timedelta(days=int(f"{period[:-1]}")) 
        elif 'm' in period:
            time_later=df.index[-1] + datetime.timedelta(minutes=int(f"{period[:-1]}")) 
        else:
            assert False, '일봉과 분봉만 조회 가능합니다.'
        
        # pd.to_datetime는 utc기준으로 date를 반환함
        btc_ohlcv = binance.fetch_ohlcv(coin_name,period,since=date_to_timestamp(time_later,utc=True))

        df=pd.DataFrame(btc_ohlcv,columns=['datetime','open','high','low','close','volume'])
        df['datetime']=pd.to_datetime(df['datetime'],unit='ms')
        df.set_index('datetime',inplace=True)
        
        while len(df)==0:  # 정기 점검이 있는 시간대에는 조회를 해도 결과가 나오지 않음. 500개씩 조회되므로 데이터가 조회될때까지 500개씩 건너뛰기
            if 'd' in period:
                time_later=time_later + datetime.timedelta(days=int(f"{period[:-1]}*500")) 
            elif 'm' in period:
                time_later=time_later + datetime.timedelta(minutes=int(f"{period[:-1]}*500")) 
            else:
                assert False, '일봉과 분봉만 조회 가능합니다.'
            
            btc_ohlcv = binance.fetch_ohlcv(coin_name,period,since=date_to_timestamp(time_later,utc=True))

            df=pd.DataFrame(btc_ohlcv,columns=['datetime','open','high','low','close','volume'])
            df['datetime']=pd.to_datetime(df['datetime'],unit='ms')
            df.set_index('datetime',inplace=True)
            
        total_df = pd.concat([total_df,df])
        
    total_df.index = total_df.index + datetime.timedelta(hours = 9)
    total_df = total_df[:end_time]
    
    coin_name = "".join(coin_name.split("/"))
    s_time = "-".join("-".join(str(start_time).split(" ")).split(":"))
    e_time = "_".join("-".join(str(end_time).split(" ")).split(":"))
    total_df.to_csv(f'./{coin_name}_{period}_{s_time}_{e_time}.csv')

In [4]:
make_csv_data(coin_name= 'BTC/USDT', period= '1m', start_time= '2022-12-01 00:00:00', end_time= '2022-12-31 23:59:00')

2022-12-04 02:19:00
2022-12-07 13:39:00
2022-12-11 00:59:00
2022-12-14 12:19:00
2022-12-17 23:39:00
2022-12-21 10:59:00
2022-12-24 22:19:00
2022-12-28 09:39:00
2022-12-31 20:59:00
